<a href="https://colab.research.google.com/github/Andyfer004/P1-DS/blob/main/P1_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Universidad del Valle de Guatemala**

## **Facultad de Ingeniería**  
### **Departamento de Ciencias de la Computación**


## **Curso: Data Science**  
### **Profesor: Luis Furlán**


## **Proyecto 1 Data Science: Limpieza de datos**  
### **Autores:**
- **Andy Fuentes 22944**
- **Davis Roldán 22672**
- **Gabriel Paz 221087**


In [9]:
# Cargar librerías y datos crudos
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

df = pd.read_csv('dataset_institutions.csv')

# Descripción del conjunto de datos crudos
filas, vars_ = df.shape
print(f"Filas: {filas}")
print(f"Variables: {vars_}")

# Vista previa
display(df.head(5))

Filas: 6607
Variables: 17


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ


- El conjunto tiene **`filas`** filas y **`vars_`** variables.  
- No se observan filas totalmente vacías.  
- Algunas columnas muestran tipos mixtos (e.g., números y strings en `TELEFONO`).  
- La vista previa revela formatos inconsistentes (mayúsculas vs minúsculas) en nombres de establecimientos.

## 2. Variables que más operaciones de limpieza necesitarán

A continuación calculamos cuántos valores nulos hay por variable y cuántos valores únicos tiene cada una.


In [3]:
# Conteo de valores nulos por variable
null_counts = df.isnull().sum().sort_values(ascending=False)
display(null_counts.head(10))

# Conteo de valores únicos por variable
unique_counts = df.nunique().sort_values()
display(unique_counts.head(10))


TELEFONO     53
DIRECTOR     33
DIRECCION     9
SECTOR        7
PLAN          7
JORNADA       7
MODALIDAD     7
STATUS        7
AREA          7
CODIGO        7
dtype: int64

NIVEL              1
STATUS             1
MODALIDAD          2
AREA               3
SECTOR             4
JORNADA            6
PLAN              12
DEPARTAMENTO      23
DEPARTAMENTAL     26
MUNICIPIO        343
dtype: int64

**Top 5 variables con más valores nulos**  
1. `TELEFONO` – 53 nulos  
2. `DIRECTOR` – 33 nulos  
3. `DIRECCION` – 9 nulos  
4. `SECTOR` – 7 nulos  
5. `PLAN` – 7 nulos  

**Otras variables a vigilar** (pocos valores únicos, posibles categorías mal codificadas):  
- `NIVEL`, `STATUS`, `MODALIDAD`, `AREA`, …


## Estrategia de Limpieza de Datos

### 1. Eliminación de espacios innecesarios
Se eliminan los espacios en blanco al inicio y al final de los valores en todas las columnas para evitar problemas al comparar, agrupar o buscar datos.

### 2. Normalización de texto
Se convierten a mayúsculas las columnas categóricas como:
- `DEPARTAMENTO`
- `MUNICIPIO`
- `ESTABLECIMIENTO`
- `SUPERVISOR`
- `DIRECTOR`
- `NIVEL`
- `SECTOR`
- `AREA`
- `STATUS`
- `MODALIDAD`
- `JORNADA`
- `PLAN`
- `DEPARTAMENTAL`

Esto asegura consistencia y evita diferencias por mayúsculas/minúsculas.

### 3. Eliminación de caracteres especiales
Se eliminan caracteres como comillas simples y dobles en nombres de establecimientos y otras columnas para evitar errores en el procesamiento de texto.

### 4. Validación y limpieza de teléfonos
Se transforman los números de teléfono a cadenas que contengan únicamente dígitos (`0-9`), eliminando espacios, guiones u otros caracteres.

### 5. Eliminación de duplicados
Se eliminan registros repetidos para asegurar la integridad de los datos.

### 6. Tratamiento de valores nulos
Se reemplazan los valores faltantes por la palabra **"DESCONOCIDO"** para evitar errores en análisis posteriores.

### 7. Creación de columnas derivadas
Se generan nuevas columnas a partir de la columna `CODIGO`:
- `CODIGO_DEPTO`: extrae el código del departamento.
- `CODIGO_MUNI`: extrae el código del municipio.

Esto permite una segmentación y análisis más detallados.

### 8. Estandarización general
Se asegura que todos los datos tengan un formato consistente, lo cual facilita futuras etapas de análisis y visualización.

### 9. Detección y manejo de valores atípicos (outliers)
Revisar columnas como TELEFONO para detectar valores fuera de rango (por ejemplo, teléfonos con más o menos de 8 dígitos).

En campos categóricos como JORNADA o MODALIDAD, verificar si hay valores poco frecuentes o incorrectos.


### 10. Codificación de variables
Convertir variables categóricas en formato numérico o de etiqueta para facilitar análisis posteriores (ejemplo: SECTOR: PRIVADO → 1, OFICIAL → 0).


## DESARROLLO DE LIMPIEZA

## 1. Eliminación de espacios innecesarios


In [ ]:
df.columns = df.columns.str.strip()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

## 2. Normalización de texto

In [14]:

cols_to_upper = [
    "DEPARTAMENTO", "MUNICIPIO", "ESTABLECIMIENTO", "SUPERVISOR", "DIRECTOR",
    "NIVEL", "SECTOR", "AREA", "STATUS", "MODALIDAD", "JORNADA", "PLAN", "DEPARTAMENTAL"
]
df[cols_to_upper] = df[cols_to_upper].apply(lambda col: col.str.upper())

## # 3. Eliminación de caracteres especiales


In [15]:

df = df.replace({r'\"': '', r"\'": ""}, regex=True)

## 4. Validación y limpieza de teléfonos

In [16]:
df["TELEFONO"] = df["TELEFONO"].astype(str).str.replace(r'\D', '', regex=True)


## 5. Detección y manejo de valores atípicos (outliers)
### 5.1 Teléfonos inválidos (no tienen 8 dígitos o no son numéricos)


In [17]:
df["TELEFONO_INVALIDO"] = df["TELEFONO"].apply(lambda x: len(x) != 8 or not x.isdigit())
print(f" Teléfonos inválidos detectados: {df['TELEFONO_INVALIDO'].sum()}")

 Teléfonos inválidos detectados: 2582


### 5.2 Outliers en columnas categóricas (valores poco frecuentes)

In [18]:
categorical_columns = ["SECTOR", "AREA", "STATUS", "MODALIDAD", "JORNADA"]
for col in categorical_columns:
    value_counts = df[col].value_counts(normalize=True)
    rare_values = value_counts[value_counts < 0.01].index  # Menos del 1% de frecuencia
    if not rare_values.empty:
        print(f"⚠ Valores poco frecuentes detectados en '{col}': {list(rare_values)}")
        df[col] = df[col].replace(list(rare_values), "DESCONOCIDO")

⚠ Valores poco frecuentes detectados en 'SECTOR': ['DESCONOCIDO']
⚠ Valores poco frecuentes detectados en 'AREA': ['DESCONOCIDO']
⚠ Valores poco frecuentes detectados en 'STATUS': ['DESCONOCIDO']
⚠ Valores poco frecuentes detectados en 'MODALIDAD': ['DESCONOCIDO']
⚠ Valores poco frecuentes detectados en 'JORNADA': ['DESCONOCIDO']


## 6. Eliminación de duplicados

In [19]:
df = df.drop_duplicates()

## 7. Tratamiento de valores nulos y valores inválidos

In [20]:
df = df.fillna("DESCONOCIDO")
df.replace(["SIN ESPECIFICAR"], "DESCONOCIDO", inplace=True)

## 8. Creación de columnas derivadas

In [21]:
df["CODIGO_DEPTO"] = df["CODIGO"].str.split("-").str[0]
df["CODIGO_MUNI"] = df["CODIGO"].str.split("-").str[1]

df["CODIGO_DEPTO"] = pd.to_numeric(df["CODIGO_DEPTO"], errors="coerce").fillna(0).astype(int)
df["CODIGO_MUNI"] = pd.to_numeric(df["CODIGO_MUNI"], errors="coerce").fillna(0).astype(int)

## 9. Codificación de variables categóricas

In [22]:
encoder = LabelEncoder()
for col in categorical_columns:
    if df[col].notna().any():
        df[col + "_ENCODED"] = encoder.fit_transform(df[col].astype(str))

## 10. Normalización adicional de texto

In [23]:
text_columns = ["ESTABLECIMIENTO", "DIRECCION", "SUPERVISOR", "DIRECTOR"]
for col in text_columns:
    df[col] = df[col].str.replace(r'\s+', ' ', regex=True)

## 11. Reordenar columnas

In [24]:
ordered_columns = [
    "CODIGO", "CODIGO_DEPTO", "CODIGO_MUNI", "DISTRITO", "DEPARTAMENTO", "MUNICIPIO",
    "ESTABLECIMIENTO", "DIRECCION", "TELEFONO", "TELEFONO_INVALIDO",
    "SUPERVISOR", "DIRECTOR", "NIVEL", "SECTOR", "SECTOR_ENCODED",
    "AREA", "AREA_ENCODED", "STATUS", "STATUS_ENCODED",
    "MODALIDAD", "MODALIDAD_ENCODED", "JORNADA", "JORNADA_ENCODED",
    "PLAN", "DEPARTAMENTAL"
]
df = df[[col for col in ordered_columns if col in df.columns]]

# 12. Reporte de valores nulos finales

In [25]:
print("\n Valores nulos después de limpieza:")
print(df.isnull().sum())


 Valores nulos después de limpieza:
CODIGO               0
CODIGO_DEPTO         0
CODIGO_MUNI          0
DISTRITO             0
DEPARTAMENTO         0
MUNICIPIO            0
ESTABLECIMIENTO      0
DIRECCION            0
TELEFONO             0
TELEFONO_INVALIDO    0
SUPERVISOR           0
DIRECTOR             0
NIVEL                0
SECTOR               0
SECTOR_ENCODED       0
AREA                 0
AREA_ENCODED         0
STATUS               0
STATUS_ENCODED       0
MODALIDAD            0
MODALIDAD_ENCODED    0
JORNADA              0
JORNADA_ENCODED      0
PLAN                 0
DEPARTAMENTAL        0
dtype: int64


## Guardar dataset limpio


In [26]:

df.to_csv("P1_DS_limpio.csv", index=False)

print("\n Limpieza completada. Archivo guardado como P1_DS_limpio.csv")



 Limpieza completada. Archivo guardado como P1_DS_limpio.csv
